# Parameter Optimization #

This project provides an easy to use functionality to implement and evaluate automatic stock trading strategies. It is implemented in java and therefore can be used in any environment which builds on the JVM.

It provides the following functionality:
- Simple access to stock data
- Declarative formulation of trading strategies
- Evaluation of trading strategies
- Optimization of trading strategies
- Support of portfolio of multiple stocks / trading strategies

In this document we describe the functionality which helps us to optimize the parameters for trading strategies.

## Setup and Imports ##
First you need to install the java libraries:

In [1]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:0.9-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT

// our stock evaluation framwork
import ch.pschatzmann.dates._;
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.data.universe._;
import ch.pschatzmann.stocks.input._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution._;
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.execution.price._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.strategy.optimization._;
import ch.pschatzmann.stocks.strategy.allocation._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.integration._;
import ch.pschatzmann.stocks.integration.ChartData.FieldName._;
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._;

// java
import java.lang._;
import java.util.ArrayList

// ta4j
import org.ta4j.core._;
import org.ta4j.core.analysis._;
import org.ta4j.core.analysis.criteria._;
import org.ta4j.core.indicators._;
import org.ta4j.core.indicators.helpers._;
import org.ta4j.core.trading.rules._;

/// jupyter custom displayer
import ch.pschatzmann.display.Displayers

Added new repo: maven-public


import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.integration.ChartData.FieldName._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._
import java.lang._
impo...

### Logging ###
The framework is using log4j. For this demonstration session we deactivate the information messages and therfore define the log level for this session to display only Errors:

In [2]:
Displayers.setup("ERROR")


true

## Trading Strategy Parameter Optimization ##

Each trading strategy can have some build-in parameters. The Optimizer is used to optimize 
the parameter values of the trading strategy to give the best indicated KPI value. In the example below 
we maximize the AbsoluteReturn.

We currently provide the following implementations for Optimizers:
- BinarySearchOptimizer
- PermutatedBinarySearchOptimizer
- SequenceOptimizer
- BruteForceOptimizer
- SimmulatedAnnealingOptimizer
- GeneticOptimizer

The BinarySearchOptimizer is using on the parameter sequence which is defined in the trading strategy to
optimize the target. In order to avoid local minima - in the case if parameters are dependent on each other - we have implemented the PermutatedBinarySearchOptimizer which is using the result of the best sequence combination. 

In [3]:
val target =  KPI.AbsoluteReturn;
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var trader = new PaperTrader(account);
var optimizer = new BinarySearchOptimizer(new Fitness(trader), KPI.AbsoluteReturn);
var result = optimizer.optimize(new RSI2Strategy(stockdata),account.getDateRange());

// print one parameter
println("Absolute Return: "+result.result().getValue(target));

// print all parameters
Displayers.display(result.getMap());

Absolute Return: 91268.0


Key,Value
PurchasedValue,112923
SharpeRatio,1
RealizedGains,12973
UnrealizedGains,78345
LongSMAPeriod,250
Cash,12
MaxDrawDownLowValue,106823
NumberOfTrades,5
NumberOfTradedStocks,1
EntryLimit,3


### Avoid Look Ahead Bias: OptimizedStrategy ###
The example above has a big problem: It suffers from look ahead bias!. This is bias created by the use of information or data that would not have been known or available during the period being analyzed. This will  lead to unrealistic results.

Therefore we need to make sure that when we create the optimization, we only use the data backward in time.
We have created the OptimizedStrategy class to help us with this. 

Here is the example that only uses the historic data up to the "2014-12-31" to optimize the strategy parameters:

In [4]:
var periods = Context.getDateRanges("2015-01-01","2016-01-01");
var account = new Account("Simulation","USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var strategy = new RSI2Strategy(stockdata);
var trader = new PaperTrader(account);
var optimizer = new BinarySearchOptimizer(new SimulatedFitness(account), KPI.AbsoluteReturn);
var optimizedStrategy = new OptimizedStrategy(strategy, optimizer, periods.get(0));                                             
var state = new Fitness(trader).getFitness(optimizedStrategy, periods.get(1));

"Return: " + state.result().getValue(KPI.AbsoluteReturn);


Return: 47198.0


We also support a more dynamic approach where we run the optimization while we execute the strategy. 
In the following example we run the optimization every month on past data:


In [5]:
var periods = Context.getDateRanges("2015-01-01","2016-01-01");
var account = new Account("Simulation","USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var strategy = new RSI2Strategy(stockdata);
var trader = new PaperTrader(account);
var optimizer = new BinarySearchOptimizer(new SimulatedFitness(account),KPI.AbsoluteReturn);
var optimizedStrategy = new OptimizedStrategy(strategy, optimizer, MONTH);                                             
var state = new Fitness(trader).getFitness(optimizedStrategy, periods.get(1));

"Return: " + state.result().getValue(KPI.AbsoluteReturn);


Return: 69346.0

### Performance of Optimizers ###

Finally we compare the result of the different Optimizers. First we run them with the standard settings:

In [6]:

def evaluateNoOptimization() = {
    val rec = new java.util.TreeMap[String,Any]();
    val start = System.currentTimeMillis()
    account.reset();
    val strategy = new RSI2Strategy(stockdata);
    var state = new SimulatedFitness(account).getFitness(strategy,account.getDateRange());
    val end = System.currentTimeMillis()
    rec.put("Strategy", "Not optimized");
    rec.put(target.name(), state.result.getValue(target));
    rec.put("Runtime (sec)", (end - start) / 1000.0);
    result.add(rec);
}

def evaluateOptimizer(optimizer : IOptimizer, name:String) = {
    println("processing "+name)
    val rec = new java.util.TreeMap[String,Any]()
    val start = System.currentTimeMillis()
    account.reset();
    
    var resultOfOptimization = optimizer.optimize(new RSI2Strategy(stockdata),account.getDateRange());
    val end = System.currentTimeMillis()
    rec.put("Strategy", name);
    rec.put(target.name(), resultOfOptimization.result().getValue(target));
    rec.put("Runtime (sec)", (end - start) / 1000.0);
    result.add(rec);
}

val result = new ArrayList[java.util.Map[String,_]]();
val target =  KPI.AbsoluteReturn;
val stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var account = new Account("Simulation","USD", 100000.00, Context.date("2015-01-01"), new PerTradeFees(10.0));
var fitness = new SimulatedFitness(account);

evaluateNoOptimization();
evaluateOptimizer(new BinarySearchOptimizer(fitness, target),"BinarySearchOptimizer");
evaluateOptimizer(new SequenceOptimizer(fitness, target),"SequenceOptimizer");
evaluateOptimizer(new SimulatedAnnealingOptimizer(fitness, target),"SimmulatedAnnealingOptimizer");
evaluateOptimizer(new GeneticOptimizer(fitness, target),"GeneticOptimizer");
// this is very slow
//evaluateOptimizer(new PermutatedBinarySearchOptimizer(fitness,target),"PermutatedBinarySearchOptimizer");

Displayers.display(result);

processing BinarySearchOptimizer
processing SequenceOptimizer
processing SimmulatedAnnealingOptimizer
processing GeneticOptimizer


AbsoluteReturn,Runtime (sec),Strategy
56741,1.746,Not optimized
91268,3.539,BinarySearchOptimizer
84676,24.626,SequenceOptimizer
94714,19.972,SimmulatedAnnealingOptimizer
113990,51.204,GeneticOptimizer


We can also influence the performance of the optimizers by setting their specific parameters:

In [7]:
result.clear()

var optimizer = new SimulatedAnnealingOptimizer(fitness,target);
optimizer.setCount(20)
evaluateOptimizer(optimizer,"SimulatedAnnealingOptimizer")

Displayers.display(result)

processing SimulatedAnnealingOptimizer


AbsoluteReturn,Runtime (sec),Strategy
90616,4.228,SimulatedAnnealingOptimizer


In [8]:
result.clear()

var optimizer = new GeneticOptimizer(fitness, target);
optimizer.setGenerations(20)
evaluateOptimizer(optimizer,"GeneticOptimizer")

Displayers.display(result)

processing GeneticOptimizer


AbsoluteReturn,Runtime (sec),Strategy
94714,106.077,GeneticOptimizer


The BinarySearchOptimizer gives the quickest results. 
The other optimizers are slow but potentilly give better results.

### Selection of the Best Strategy for a Stock ###

In the examples above we were optimizing the performance of a selected strategy. 
Next we show how to find the best (not optimized) strategy.



In [9]:
var strategies = TradingStrategyFactory.list()
var periods = Context.getDateRanges("2015-01-01","2016-01-01");
var account = new Account("Simulation","USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var trader = new PaperTrader(account);
var state = new StrategySelector(account, strategies, periods.get(0), KPI.AbsoluteReturn).getMax(stockdata)

"Return: " + state.result().getValue(KPI.AbsoluteReturn) +" with "+ state.getStrategyName();

Return: 81524.0 with GlobalExtremaStrategy

### Selection of the Best Optimized Strategy ###

Finally we show how to find the best optimized strategy.


In [10]:
var periods = Context.getDateRanges("2015-01-01","2016-01-01");
var strategies = TradingStrategyFactory.list()
var account = new Account("Simulation","USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var stockdata = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var trader = new PaperTrader(account);
var optimizer = new BinarySearchOptimizer(new SimulatedFitness(account),KPI.AbsoluteReturn);
var state = new StrategySelectorOptimized(account, strategies, 
    periods.get(0),
    periods.get(1),
    new BinarySearchOptimizer(new SimulatedFitness(account),KPI.AbsoluteReturn))
   .getMax(stockdata)

"Return: " + state.result().getValue(KPI.AbsoluteReturn) +" with "+ state.getStrategyName();

Return: 75819.0 with BuyAndHoldStrategy